In [1]:
# import giskard

import warnings
import logging

import numpy as np
import pandas as pd
import pickle

from functions.loading import load_data

from functions.training_pipeline import training_pipeline
from functions.models import  catboost_model

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

warnings.filterwarnings("ignore")
pd.options.mode.chained_assignment = None

pd.set_option('display.max_columns', 500)

In [2]:
# Training parameters
restricted_features=False
save=True


models = {
        "catboost": catboost_model,
}

targets = ["cf1", "cf2", "cf3", "cf123"]
# targets = ["cf1"]

# Results containers
summary_final=[]
summary_metrics_detailed = pd.DataFrame()
estimated_scopes = []

In [3]:
# predict_dataset = load_data(save=True, filter_outliers=False)


In [4]:
preprocessed_dataset = load_data(save=True)



cf1 (outliers preprocess): enough observations : 142
cf1 (outliers preprocess): not enough observations : 2
Number of observations dropped: 2756
cf2 (outliers preprocess): enough observations : 142
cf2 (outliers preprocess): not enough observations : 2
Number of observations dropped: 2045
cf3 (outliers preprocess): enough observations : 142
cf3 (outliers preprocess): not enough observations : 2
Number of observations dropped: 1081
cf123 (outliers preprocess): enough observations : 142
cf123 (outliers preprocess): not enough observations : 2
Number of observations dropped: 587


In [5]:
best_scores, best_stds, summary_global, summary_metrics_detailed = training_pipeline(
    targets=targets,
    models=models,
    summary_final=summary_final,
    summary_metrics_detailed=summary_metrics_detailed,
    estimated_scopes=estimated_scopes,
    preprocessed_dataset=preprocessed_dataset,
    restricted_features=restricted_features,
    save=save,
)

INFO:functions.training_pipeline:Training for target: cf1


Files not found, constructing them
Constructing benchmark
train before imputation :  (626577, 68)
test before imputation :  (28015, 68)
train 1 :  (626577, 1161)
test 1 :  (28015, 741)
train 2 :  (626577, 1162)
test 2 :  (28015, 757)
train avant target_preprocessing :  (626577, 1162)
test avant target_preprocessing :  (28015, 757)


INFO:functions.training_pipeline:Preprocessing done
INFO:functions.training_pipeline:Training model: catboost


train :  (19934, 1162)
test :  (10662, 757)
preprocessed :  (654592, 68)


INFO:functions.training_pipeline:Modelisation done
INFO:functions.training_pipeline:Elapsed time for target cf1: 2129.83 seconds


cf1
  Target     model       mae       mse        r2      rmse      mape     std
0    cf1  catboost  0.346437  0.226323  0.882573  0.475734  0.593453  0.0656
-------------------------
[0.47573388985311804]
